In [1]:
import numpy as np
import numpy as np
from concrete import fhe
from Bio.Seq import Seq, MutableSeq

import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

from concrete_biopython.FheSeq import FheSeq, FheMutableSeq
from concrete_biopython.SeqWrapper import SeqWrapper


In [7]:
fhe.zeros(0)

ValueError: zero-size array to reduction operation minimum which has no identity

In [2]:
seq1 = Seq('ACCAGGTAC')
seq2 = MutableSeq('CGTTAGC')
seq = seq1+seq2

In [3]:
def process_seq(seq):
    seq1=seq[0:9]
    seq2=FheMutableSeq(seq[9:len(seq)])
    seq2.pop()
    new_seq = seq1.reverse_complement() + seq2[0:3]
    protein = new_seq.translate('Standard')
    return protein

In [4]:
# wrap a fhe circuit in order to input and output Bio.Seq objects.
def circuit_wrapper(circuit, seq, simulate=False):
    # convert Seq objects to integers with SeqWrapper.toIntegers
    integers = SeqWrapper(seq).toIntegers()
    # run the circuit with integer inputs
    integer_output = circuit.simulate(integers) if simulate else circuit.encrypt_run_decrypt(integers)
    # convert back the integer outputs into a Seq objects with SeqWrapper.toSeq
    return SeqWrapper(integer_output).toSeq()

In [5]:
def process_seq_adapter(integer_seq,_):
    # convert integer to FheSeq object, process it and return result
    return process_seq(FheSeq(integer_seq)).toArray()

In [6]:
# compile the process_seq_adapter function and create a circuit
compiler = fhe.Compiler(lambda data1,data2: process_seq_adapter(data1, data2), {"data1": "encrypted", "data2": "encrypted"})
circuit = compiler.compile(
    inputset=[
    (np.random.randint(0, SeqWrapper.maxInteger()+1, size=(len(seq1),)),
    np.random.randint(0, SeqWrapper.maxInteger()+1, size=(len(seq2),)))
    for _ in range(100)
    ],
    configuration=fhe.Configuration(
        enable_unsafe_features=True,
        use_insecure_key_cache=True,
        insecure_key_cache_location=".keys",
        dataflow_parallelize=False, # setting it to True makes the jupyter kernel crash
    ),
    verbose=False,
)

ValueError: zero-size array to reduction operation minimum which has no identity